# XmR Chart Example

This notebook demonstrates the usage of the spc_plotly package with sample vehicle count data.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from spc_plotly import xmr

# Set random seed for reproducibility
np.random.seed(475)

## Make data

In [2]:
# Generate 100 days of data
start_date = datetime(2024, 1, 1)
dates = [start_date + timedelta(days=x) for x in range(100)]

# Generate vehicle counts with some patterns:
# - Base level around 1000 vehicles
# - Gradual upward trend
# - Some random variation
# - A few anomalies

base_counts = 1000
trend = np.linspace(0, 200, 100)  # Gradual increase over time
variation = np.random.normal(0, 50, 100)  # Random daily variation
weekly_pattern = np.tile([50, 30, 0, -20, -30, -80, -100], 15)[:100]  # Weekly patterns

# Combine components
vehicle_counts = base_counts + trend + variation + weekly_pattern

# Add some anomalies
vehicle_counts[25] += 300  # Special event
vehicle_counts[50:55] += 200  # Construction period
vehicle_counts[80] -= 400  # Road closure

# Create DataFrame
data = pd.DataFrame({
    'date': dates,
    'vehicle_counts': vehicle_counts.astype(int)
})

# Display first few rows
data.head()

,date,vehicle_counts
0,2024-01-01,942
1,2024-01-02,1033
2,2024-01-03,1023
3,2024-01-04,983
4,2024-01-05,1011


## Date example

In [3]:
# Create XmR chart
xmr_chart = xmr.XmR(
    data=data,
    x_type='date_time',
    x_ser_name='date',
    y_ser_name='vehicle_counts',
    date_part_resolution='day',
    title='Daily Vehicle Counts - XmR Chart',
    period_breaks = ['2024-01-21', '2024-03-01']
)

# Display the chart
xmr_chart.xmr_chart

In [4]:
# Check the calculated limits and signals
print("Moving Range Limits:")
print(xmr_chart.mR_limit_values)
print("\nNatural Process Limits:")
print(xmr_chart.npl_limit_values)
print("\nDetected Signals:")
print(xmr_chart.signals)

Moving Range Limits:
{'mR_xmr_func': [np.float64(85.75), np.float64(104.025), np.float64(80.82051282051282)], 'mR_upper_limit': [np.float64(280.231), np.float64(339.95369999999997), np.float64(264.12143589743584)], 'xmr_func': 'mean'}

Natural Process Limits:
{'y_xmr_func': [np.float64(980.4285714285714), np.float64(1098.5121951219512), np.float64(1123.675)], 'npl_upper_limit': [np.float64(1208.5235714285714), np.float64(1375.2186951219512), np.float64(1338.657564102564)], 'npl_lower_limit': [np.float64(752.3335714285714), np.float64(821.8056951219512), np.float64(908.6924358974359)], 'xmr_func': 'mean'}

Detected Signals:
{'anomalies': [(datetime.datetime(2024, 1, 26, 0, 0), np.int64(1340), 'High'), (datetime.datetime(2024, 2, 12, 0, 0), np.int64(1229), 'High'), (datetime.datetime(2024, 2, 20, 0, 0), np.int64(1217), 'High'), (datetime.datetime(2024, 2, 21, 0, 0), np.int64(1317), 'High'), (datetime.datetime(2024, 2, 22, 0, 0), np.int64(1315), 'High'), (datetime.datetime(2024, 2, 23, 0,

In [8]:
# Numeric example
numeric_data = pd.DataFrame({
    'position': range(1, 101),
    'measurement': np.random.normal(100, 10, 100)
})

xmr_numeric = xmr.XmR(
    data=numeric_data,
    x_ser_name='position',
    y_ser_name='measurement',
    x_type='numeric',
    chart_height=800,
    period_breaks=[40, 45],
)

xmr_numeric.xmr_chart

In [6]:
# Categorical example
categories = ['Station_' + chr(ord('a') + i) for i in range(100)]
categorical_data = pd.DataFrame({
    'station': categories,
    'quality_score': np.random.normal(85, 5, 100)
}).sort_values('station')

xmr_categorical = xmr.XmR(
    data=categorical_data,
    x_ser_name='station',
    y_ser_name='quality_score',
    x_type='categorical',
    # show_grid=False
)

xmr_categorical.xmr_chart